# Agent Short-Term Memory

Short-term memory stores conversation history within a session using checkpointers.

**What you'll learn:**
- Checkpointers persist conversation history
- SQLite for development, PostgreSQL for production
- Thread IDs manage separate sessions
- Access agent state in tools with ToolRuntime
- Modify agent state from tools for context offloading
- Save/load conversation summaries for long conversations

## Checkpointer Comparison

| Type | Use Case | Setup |
|------|----------|-------|
| **SQLite** | Development, testing | Simple file-based |
| **PostgreSQL** | Production, multi-user | Database connection |

In [1]:
import sys
sys.path.append('../')

import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import create_agent
from langchain.messages import HumanMessage, ToolMessage, SystemMessage
from langchain.tools import tool, ToolRuntime
from pydantic import BaseModel
from pathlib import Path
from langgraph.types import Command
from scripts import base_tools

In [3]:
model = ChatGoogleGenerativeAI(model='gemini-2.5-flash')

## Problem: No Memory

In [4]:
agent = create_agent(model=model)

agent.invoke({'messages': [HumanMessage("My name is John")]})
response = agent.invoke({'messages': [HumanMessage("What's my name?")]})

response['messages'][-1].text

"As an AI, I don't have access to personal information about you, including your name. I don't know who you are.\n\nIf you'd like me to refer to you by a specific name during our conversation, feel free to tell me!"

## Short-Term Memory: SQLite

In [5]:
from langgraph.checkpoint.sqlite import SqliteSaver
import sqlite3

os.makedirs("db", exist_ok=True)

conn = sqlite3.connect("db/checkpoints.db", check_same_thread=False)
checkpointer = SqliteSaver(conn)

agent = create_agent(
    model=model,
    tools=[base_tools.web_search, base_tools.get_weather],
    checkpointer=checkpointer
)

In [6]:
config = {"configurable": {"thread_id": "user_123"}}

agent.invoke({'messages': [HumanMessage("My name is John")]}, config)
response = agent.invoke({'messages': [HumanMessage("What's my name?")]}, config)

response['messages'][-1].text

"I'm sorry, I don't have memory of past conversations. Could you please tell me your name again?"

In [7]:
response

{'messages': [HumanMessage(content='My name is John', additional_kwargs={}, response_metadata={}, id='41f29461-4ba2-4ec3-8e3d-94a88e10acc1'),
  AIMessage(content='Hello John! How can I help you today?', additional_kwargs={}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019ba176-c2ad-7450-b4ae-9803ec00dd25-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 206, 'output_tokens': 10, 'total_tokens': 216, 'input_token_details': {'cache_read': 0}}),
  HumanMessage(content="What's my name?", additional_kwargs={}, response_metadata={}, id='a5896699-f58b-4611-9b78-d155793343e0'),
  AIMessage(content="I'm sorry, I don't have memory of past conversations. Could you please tell me your name again?", additional_kwargs={}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019ba176

## Short-Term Memory: PostgreSQL

In [ ]:
# from langgraph.checkpoint.postgres import PostgresSaver
# import psycopg

# with PostgresSaver.from_conn_string(os.getenv("POSTGRESQL_URL")) as checkpointer:
#     agent = create_agent(
#         model=model,
#         tools=[base_tools.web_search, base_tools.get_weather],
#         checkpointer=checkpointer
#     )
    
#     config = {"configurable": {"thread_id": "postgres_session"}}
    
#     agent.invoke({'messages': [HumanMessage("My city is Mumbai")]}, config)
#     response = agent.invoke({'messages': [HumanMessage("What's my city?")]}, config)
    
#     response['messages'][-1].text

## Context Offloading: Read State in Tools

Access agent state to save conversation summaries for context management.

In [8]:
class CustomContext(BaseModel):
    user_id: str
    thread_id: str

@tool
def save_conversation_summary(summary: str, runtime: ToolRuntime[CustomContext]) -> str:
    """Save conversation summary to disk for context offloading."""
    user_id = runtime.context.user_id
    thread_id = runtime.context.thread_id
    
    # Create directory structure
    summary_dir = Path(f"data/{user_id}/{thread_id}")
    summary_dir.mkdir(parents=True, exist_ok=True)
    
    # Write summary
    summary_path = summary_dir / "summary.md"
    summary_path.write_text(summary)
    
    return f"Summary saved to {summary_path}"

In [10]:
# Test save summary
agent = create_agent(
    model=model,
    tools=[save_conversation_summary],
    checkpointer=checkpointer,
    context_schema=CustomContext
)

config = {"configurable": {"thread_id": "session1"}}

response = agent.invoke({
    "messages": [HumanMessage("Save this summary: User discussed Python and AI topics")]
}, config=config, context=CustomContext(user_id="user_123", thread_id="session1"))

response['messages'][-1].text

'I have saved the summary of our conversation.'

## Context Offloading: Modify State in Tools

Load previous summaries and inject them into agent state.

In [11]:
@tool
def load_conversation_summary(runtime: ToolRuntime[CustomContext]) -> Command:
    """Load previous conversation summary from disk."""
    user_id = runtime.context.user_id
    thread_id = runtime.context.thread_id
    
    summary_path = Path(f"data/{user_id}/{thread_id}/summary.md")
    
    if not summary_path.exists():
        return Command(update={
            "messages": [
                ToolMessage(
                    "No previous summary found.",
                    tool_call_id=runtime.tool_call_id
                )
            ]
        })
    
    # Read summary
    summary_text = summary_path.read_text()
    
    # Update state with summary as system message
    return Command(update={
        "messages": [
            SystemMessage(f"Previous conversation summary:\n{summary_text}"),
            ToolMessage(
                "Successfully loaded previous summary.",
                tool_call_id=runtime.tool_call_id
            )
        ]
    })

In [ ]:
# Test load summary
agent = create_agent(
    model=model,
    tools=[load_conversation_summary],
    checkpointer=checkpointer,
    context_schema=CustomContext
)

config = {"configurable": {"thread_id": "session1"}}

response = agent.invoke({
    "messages": [HumanMessage("Load my previous conversation summary")]
}, config=config, context=CustomContext(user_id="user_123", thread_id="session1"))

In [13]:
response

{'messages': [HumanMessage(content='Save this summary: User discussed Python and AI topics', additional_kwargs={}, response_metadata={}, id='507b082c-6c50-424e-899d-f06385a03cae'),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'save_conversation_summary', 'arguments': '{"summary": "User discussed Python and AI topics"}'}, '__gemini_function_call_thought_signatures__': {'d0a8af1f-2729-4453-a7f8-45225df71b74': 'CqICAXLI2nyu/aehi5KkIa3OtfkQlp1cgOKbNBGgOLeVVLg+QwQvu02engdoc4LYoIr6yw2qKAhQ6BqyKaxJu9cQIZaOA7PzDRfMILaYeCt8oFthyTRKYNvZNhL9O9s4HmYxZOdFJpJLru977ukSJt+ir/yXSxK1t/Y/ZbuBy+KfnkbRlsacEYaQ2X5bAmNHh3/whw4Ab+U2HdsXyKI0v0Oz9fdHDpffuVFB/kPZliZwNAmYoPyx9QpmBqHdyt55aeLMKjeP6t/ZFK351Nrm6nFG8BEacoQT7jeTafTI9zIYiAK7Ywpk8dGptM/nnCcvXCm3aE5an7dzuLH2z7pwmmNv/CeBw4734RSUgjymiHHgxODgNX2o8nAdCpizPtwpVxUXpXU='}}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019ba17a-460c-76f1-b

## Combined: Save and Load Context

Real-world example combining both tools for context management.

In [14]:
# Agent with both save and load
agent = create_agent(
    model=model,
    tools=[save_conversation_summary, load_conversation_summary],
    checkpointer=checkpointer,
    context_schema=CustomContext
)

# Session 1: Have conversation and save summary
config = {"configurable": {"thread_id": "project_alpha"}}

agent.invoke({
    "messages": [HumanMessage("We're building a chatbot using LangChain")]
}, config=config, context=CustomContext(user_id="user_456", thread_id="project_alpha"))

agent.invoke({
    "messages": [HumanMessage("Save summary: Building LangChain chatbot with memory and tools")]
}, config=config, context=CustomContext(user_id="user_456", thread_id="project_alpha"))

print("Summary saved for project_alpha")

Summary saved for project_alpha
